# All filters generator

### Parameters

In [49]:
coefficients = [
                    [ 1,  -3,  63,   4,  -2, 1],
                    [ 1,  -5,  62,   8,  -3, 1],
                    [ 2,  -8,  60,  13,  -4, 1],

                    [ 3, -10,  58,  17,  -5, 1],
                    [ 3, -11,  52,  26,  -8, 2],
                    [ 2,  -9,  47,  31, -10, 3],
                    [ 3, -11,  45,  34, -10, 3],

                    [ 3, -11,  40,  40, -11, 3],
                    [ 3, -10,  34,  45, -11, 3],
                    [ 3, -10,  31,  47,  -9, 2],
                    [ 2,  -8,  26,  52, -11, 3],

                    [ 1,  -5,  17,  58, -10, 3],
                    [ 1,  -4,  13,  60,  -8, 2],
                    [ 1,  -3,   8,  62,  -5, 1],
                    [ 1,  -2,   4,  63,  -3, 1]
                    ]

In [45]:
# N = 16              # Number of pixels by block
FRAC = 15           # Number of fractional pixels for interpolation (eg: 1/4 of resolution)
# MODE = 0            # Select MC(0) or FME(1):
# SEARCH = 2          # [Only for MC] Select Horizontal search(0), Vertical search(1) or Full search(2)
TAPS = 8            # Number of taps in the filter
# I = 8               # Input size of the line 
# S = 1               # Stride=1 means that the filter will deslocate 1 by 1 pixel at the time

PADDING = 2         # How much padding to add in each four sides

IN_NUM = 6          # 1 line of 6 samples (6-tap)
IN_BIT_WIDTH = 8    # Number of bits per sample

OUT_NUM = 15        # There are 15 filters used to generate 1/16 precision

In [13]:
import math
import numpy as np
from matplotlib import pyplot as plt

### Variables calculation

#### Input and outputs

In [48]:
# Input size
in_width = IN_NUM * IN_BIT_WIDTH

# Input text
in_text = "\tinput signed ["+str(in_width-1)+":0] X;"

# Output ports
out_names = []
for i in range(1,OUT_NUM+1):
    if i != OUT_NUM:
        out_names.append('\tY'+str(i)+',')
    else:
        out_names.append('\tY'+str(i))
out_names_concat = '\n'.join(out_names)

# Output port sizes
out_port_sizes = []
for i in range(0,OUT_NUM):
    out_port_sizes.append(18)

# Output port modes
out_port = []
for i in range(1,OUT_NUM+1):
    out_port.append('\toutput signed ['+str(out_port_sizes[i-1])+':0] Y'+str(i)+';')
out_port_concat = '\n'.join(out_port)


#### Wires

In [44]:
# Wires

#### Filter calculations

In [ ]:
modules = []
for t in range(0,TAPS):
    modules[].append

In [ ]:
t0_affine T0 (
    .X      (X[7:0]),
    .Y1     (w_t0_1),
    ...
    .Y15    (w_t0_1)
);

### Final Verilog text

In [47]:
verilog = '''/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 13/02/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
        X,
    '''+out_names_concat+'''
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

'''+in_text+'''
'''+out_port_concat+'''

// ------------------------------------------
// Wires declarations
// ------------------------------------------

//  wire signed [8:0] w2;
//  wire signed [9:0] w2_;

	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------

//  assign w1 = X;
//  assign w4 = w1 << 2;
//  assign w3 = w4 - w1;
//  assign w5 = w1 + w4;
//  assign w2_ = -1 * w2;

  
// ------------------------------------------
// Outputs
// ------------------------------------------

//  assign Y1 = w2_;


endmodule //all_filters

'''
print(verilog)

/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 13/02/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
        X,
    	Y1,
	Y2,
	Y3,
	Y4,
	Y5,
	Y6,
	Y7,
	Y8,
	Y9,
	Y10,
	Y11,
	Y12,
	Y13,
	Y14,
	Y15
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

	input signed [7:0] X;
	output signed [18:0] Y1;
	output signed [18:0] Y2;
	output signed [18:0] Y3;
	output signed [18:0] Y4;
	output signed [18:0] Y5;
	output signed [18:0] Y6;
	output signed [18:0] Y7;
	output signed [18:0] Y8;
	output signed [18:0] Y9;